In [1]:
# Package imports
import numpy as np
import os
import pandas as pd
import proplot as pplt
import sys

/opt/miniconda3/envs/drifter/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Folder with the drift tracks from 
# the Arctic Data Center (Bliss et al. 2021)
# https://arcticdata.io/catalog/view/urn%3Auuid%3A2b859a39-cada-4147-819a-dc222f6f89a3
dataloc = '../../data/adc_dn_tracks/'
saveloc = '../data/mosaic_interpolated/'# Maximum gap in hours for the interpolation algorithm

max_dt = 2

# Compute uncertainty threshold for splines?
recompute_thresh = False

# Pad for interpolation, so that the first and last values aren't missing.
margin = pd.to_timedelta('6H')
buoy_data = {}


# Optional: focus in on only a portion of the tracks
begin = '2019-10-01 00:00'
end = '2022-01-01 00:00'

# Read in the files, including the metadata file
files = os.listdir(dataloc)
files = [f for f in files if f.split('.')[-1] == 'csv']
files = [f for f in files if f.split('_')[0] != 'DN']

metadata = pd.read_csv(dataloc + 'DN_buoy_list_v2.csv')
metadata['filename'] = ['_'.join([x, str(y), z]) for 
                        x, y, z in zip(metadata['DN Station ID'],
                                       metadata['IMEI'],
                                       metadata['Sensor ID'])]
    
file_present = np.array([f + '.csv' in files for f in metadata.filename])
metadata = metadata.loc[file_present].copy()
metadata.set_index('Sensor ID', inplace=True)

buoy_data = {}
freq = []
buoys = []
for file in files:
    data = pd.read_csv(dataloc + file, index_col='datetime', parse_dates=True)
    buoy_data[buoy] = interp.
    dt = pd.to_timedelta(np.diff(data.index)).median().seconds/3600
    buoy = file.replace('.csv', '').split('_')[-1]
    if np.round(dt,1) <= max_dt:
        buoy_data[buoy] = data.copy()
        freq.append(np.round(dt, 1))
        buoys.append(buoy)
    else:
        print(buoy, dt)
buoy_freq = pd.Series(freq, buoys)

# Based on comparison with co-located sites, all the V buoys have time indices offset by 8 hours.
for buoy in ['2019V1', '2019V2', '2019V3', '2019V4', '2019V5']:
    buoy_data[buoy].index = buoy_data[buoy].index - pd.to_timedelta('8H')
# Buoys 2019P112-114 have problems that need to be cleaned individually
buoy_data = {buoy: buoy_data[buoy] for buoy in buoy_data if buoy not in ['2019P112', '2019P113', '2019P114', '2019P119']}# Folder with the drift tracks from 
# the Arctic Data Center (Bliss et al. 2021)
# https://arcticdata.io/catalog/view/urn%3Auuid%3A2b859a39-cada-4147-819a-dc222f6f89a3
dataloc = '../../data/adc_dn_tracks/'
saveloc = '../data/mosaic_interpolated/'

2020P215 3.0
2019I2 3.999722222222222
2020P214 3.0
2020P223 3.0
2019F1 3.0
2020P211 3.0
2020T60 12.0
2019I3 4.000277777777778
2019I1 3.999722222222222
2019P22 3.0
2019T47 12.000555555555556
2020P210 3.0
2019I4 4.0


In [9]:
metadata['Station Type'] = [x[0] for x in metadata['DN Station ID']]
metadata.loc[metadata['DN Station ID'] == 'LM', 'Station Type'] = 'LM'

In [42]:

data = metadata.groupby(['Manufacturer',
                  'Buoy Type',
                  'Station Type',
                  'Deployment Leg']).count()[['IMEI']].unstack('Deployment Leg').fillna(0).astype(int)
data

IMEI  \
Deployment Leg                                                                       1   
Manufacturer                        Buoy Type                        Station Type        
Bruncin                             IMBFlex                          C               0   
                                                                     L               1   
                                    Radiation Station                C               0   
                                                                     L               2   
                                                                     LM              0   
CryoInn                             Seasonal IMB                     C               1   
                                                                     L               3   
Hemisphere                          Atmospheric Surface Flux Station C               1   
                                                                     L               3   
MarlinYUG                           SVP                              P               4   
MetOcean Telematics                 SVP                              C               2   
                                                                     P              16   
                                    Snow Buoy                        C               0   
                                                                     L               3   
                                                                     LM              1   
                                                                     M               8   
                                                                     P               0   
Naval Postgraduate School           Flux Buoy                        C               1   
                                                                     L               3   
PacificGyre IT                      CTD                              C               0   
                                                                     M               8   
                                    SVP                              P              33   
SAMS Enterprise Ltd                 LITO/Envipope                    C               0   
                                                                     LM              0   
                                    Thermistor                       C               0   
                                                                     L               4   
                                                                     LM              2   
                                                                     M               8   
                                                                     P               2   
TUT                                 SVP                              P              12   
Tianjin Yuanxun Technology Co. Ltd. DTOP Ocean Profiler              M               5   

                                                                                      \
Deployment Leg                                                                     2   
Manufacturer                        Buoy Type                        Station Type      
Bruncin                             IMBFlex                          C             1   
                                                                     L             0   
                                    Radiation Station                C             0   
                                                                     L             0   
                                                                     LM            0   
CryoInn                             Seasonal IMB                     C             0   
                                                                     L             0   
Hemisphere                          Atmospheric Surface Flux Station C             0   
                                                     

In [25]:
data_C = metadata.rename({'IMEI': 'Count'}, axis=1).loc[metadata['Station Type'] == 'C'].groupby(['Manufacturer',
                  'Buoy Type',
                  'Deployment Leg']).count()[['Count']].unstack('Deployment Leg').fillna(0).astype(int)
data_L = metadata.rename({'IMEI': 'Count'}, axis=1).loc[metadata['Station Type'] == 'L'].groupby(['Manufacturer',
                  'Buoy Type',
                  'Deployment Leg']).count()[['Count']].unstack('Deployment Leg').fillna(0).astype(int)
data_M = metadata.rename({'IMEI': 'Count'}, axis=1).loc[metadata['Station Type'] == 'M'].groupby(['Manufacturer',
                  'Buoy Type',
                  'Deployment Leg']).count()[['Count']].unstack('Deployment Leg').fillna(0).astype(int)
data_P = metadata.rename({'IMEI': 'Count'}, axis=1).loc[metadata['Station Type'] == 'P'].groupby(['Manufacturer',
                  'Buoy Type',
                  'Deployment Leg']).count()[['Count']].unstack('Deployment Leg').fillna(0).astype(int)
data_LM = metadata.rename({'IMEI': 'Count'}, axis=1).loc[metadata['Station Type'] == 'LM'].groupby(['Manufacturer',
                  'Buoy Type',
                  'Deployment Leg']).count()[['Count']].unstack('Deployment Leg').fillna(0).astype(int)

In [36]:
print('n: ', data_C.sum().sum())
data_C

n:  40


Count            
Deployment Leg                                                 1  2  3  4  5
Manufacturer              Buoy Type                                         
Bruncin                   IMBFlex                              0  1  0  2  1
                          Radiation Station                    0  0  1  0  1
CryoInn                   Seasonal IMB                         1  0  0  0  0
Hemisphere                Atmospheric Surface Flux Station     1  0  2  2  4
MetOcean Telematics       SVP                                  2  0  5  0  0
                          Snow Buoy                            0  0  1  0  4
Naval Postgraduate School Flux Buoy                            1  0  0  0  0
PacificGyre IT            CTD                                  0  0  0  0  1
SAMS Enterprise Ltd       LITO/Envipope                        0  2  0  0  0
                          Thermistor                           0  2  2  0  4

In [34]:
print('n: ', data_M.sum().sum())
data_M

n:  29


,,Count
,Deployment Leg,1
Manufacturer,Buoy Type,
MetOcean Telematics,Snow Buoy,8
PacificGyre IT,CTD,8
SAMS Enterprise Ltd,Thermistor,8
Tianjin Yuanxun Technology Co. Ltd.,DTOP Ocean Profiler,5


In [37]:
print('n: ', data_LM.sum().sum())
data_LM

n:  5


Count      
Deployment Leg                            1  2  3
Manufacturer        Buoy Type                    
Bruncin             Radiation Station     0  0  1
MetOcean Telematics Snow Buoy             1  0  0
SAMS Enterprise Ltd LITO/Envipope         0  1  0
                    Thermistor            2  0  0

In [35]:
print('n: ', data_P.sum().sum())
data_P

n:  115


Count       
Deployment Leg                     1  3   5
Manufacturer        Buoy Type              
MarlinYUG           SVP            4  0   0
MetOcean Telematics SVP           16  7   8
                    Snow Buoy      0  0   2
PacificGyre IT      SVP           33  7  20
SAMS Enterprise Ltd Thermistor     2  4   0
TUT                 SVP           12  0   0